# Senate Voting Test

_James Weichert_

In [312]:
import pandas as pd
import numpy as np
import bpr

import chart_studio.plotly as pl
import plotly.graph_objects as go
from ipywidgets import widgets

In [313]:
current_senate = pd.read_csv('party_data.csv', index_col=0)
new_senate = pd.read_csv('new_senate.csv', index_col=0)

In [314]:
current_senate.head()

,State,Population (2018),U.S. House Seats,Senators,PVI,D_share,R_share
0,AL,4887871,7,2,-14,43.0,57.0
1,AK,737438,1,2,-9,45.5,54.5
2,AZ,7171646,9,2,-5,47.5,52.5
3,AR,3013825,4,2,-15,42.5,57.5
4,CA,39557045,53,2,12,56.0,44.0


In [315]:
new_senate.head()

,State,Population (2018),Senators,PVI,D_share,R_share
0,AL,4887871,1,-14,43.0,57.0
1,AK,737438,1,-9,45.5,54.5
2,AZ,7171646,2,-5,47.5,52.5
3,AR,3013825,1,-15,42.5,57.5
4,CA,39557045,12,12,56.0,44.0


In [316]:
data = current_senate

In [416]:
def simulate_election(data, turnout=0.56, turnout_variability = 0.05, voting_age_pop=0.77, swing_vote_percent=0.02, swing_factor=0.1, rand=False):
    
    data['D_sen'] = 0
    data['R_sen'] = 0

    for state in range(50): 
        
        registered_voters = int(data.loc[state, 'Population (2018)'] * voting_age_pop)
        
        for seat in range(data.loc[state, 'Senators']):
            
            if rand:
                election_turnout = turnout + (np.random.random() * 2 * turnout_variability) - turnout_variability
            else:
                election_turnout = turnout

            voters = registered_voters * election_turnout
            
            if rand:
                swing_vote_preference = (data.loc[state, 'D_share'] / (data.loc[state, 'D_sen'] + 1) - data.loc[state, 'R_share'] / (data.loc[state, 'R_sen'] + 1)) * swing_factor
            
            d_vote_share = int((voters * (1 - swing_vote_percent)) * (data.loc[state, 'D_share'] / 100) + swing_vote_preference * swing_vote_percent * voters)
            r_vote_share = int(voters - d_vote_share)
            
            if d_vote_share > r_vote_share:
                data.loc[state, 'D_sen'] += 1
            elif r_vote_share > d_vote_share:
                data.loc[state, 'R_sen'] += 1
            else:
                r = np.random.rand()
                
                if r < 0.5:
                    data.loc[state, 'D_sen'] += 1
                else:
                    data.loc[state, 'R_sen'] += 1
    
    data['Senate_Scale'] = data['R_sen'] / data['Senators']
    
    return data

In [417]:
curr_senate_voted = simulate_election(current_senate, rand=True)
curr_senate_voted.head()

,State,Population (2018),U.S. House Seats,Senators,PVI,D_share,R_share,D_sen,R_sen,Senate_Scale
0,AL,4887871,7,2,-14,43.0,57.0,0,2,1.0
1,AK,737438,1,2,-9,45.5,54.5,0,2,1.0
2,AZ,7171646,9,2,-5,47.5,52.5,1,1,0.5
3,AR,3013825,4,2,-15,42.5,57.5,0,2,1.0
4,CA,39557045,53,2,12,56.0,44.0,2,0,0.0


In [418]:
def check_composition(data):
    
    d_senate_num = np.sum(data['D_sen'])
    r_senate_num = np.sum(data['R_sen'])
    senate_total = np.sum(data['Senators'])
    return r_senate_num, d_senate_num

In [419]:
r, d = check_composition(curr_senate_voted)

In [420]:
vap = widgets.FloatSlider(
    value = 0.77,
    min=0.01,
    max=1,
    step=0.01,
    description='Voting Pop:',
    continuous_update=False
)

In [421]:
turnout = widgets.FloatSlider(
    value = 0.56,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Turnout:',
    continuous_update=False
)

In [422]:
turnout_var = widgets.FloatSlider(
    value = 0.05,
    min=0.0,
    max=1.0 - turnout.value,
    step=0.01,
    description='Turnout Var:',
    continuous_update=False
)

In [423]:
swing_vote_percentage = widgets.FloatSlider(
    value = 0.05,
    min=0.0,
    max=0.50,
    step=0.01,
    description='Swing %:',
    continuous_update=False
)

In [429]:
swing_impact = widgets.FloatSlider(
    value = 0.1,
    min=0.0,
    max=0.20,
    step=0.01,
    description='Swing Impact:',
    continuous_update=False
)

In [430]:
def response(change):
    elected_sen = simulate_election(data, turnout=turnout.value, turnout_variability=turnout_var.value, voting_age_pop=vap.value, swing_vote_percent=swing_vote_percentage.value, swing_factor=swing_impact.value, rand=True)
    r, d = check_composition(elected_sen)
    with g.batch_update():
            g.data[0].z = elected_sen['Senate_Scale']
            hovertext =  'R: ' + new_senate_elections['R_sen'].map(str) + ' D: ' + new_senate_elections['D_sen'].map(str)
            hoverinfo = 'location + text'
    g.update_layout(
        title_text = 'U.S. Senate (R: ' + str(r) + ' D: ' + str(d) + ')',
    )

In [431]:
vap.observe(response, names="value")
turnout.observe(response, names="value")
turnout_var.observe(response, names="value")
swing_vote_percentage.observe(response, names="value")
swing_impact.observe(response, names="value")

In [442]:
g = go.FigureWidget(
    data = go.Choropleth(
        locations = current_senate['State'],
        z = current_senate['Senate_Scale'].astype(float),
        hovertext =  'R: ' + current_senate['R_sen'].map(str) + ' D: ' + current_senate['D_sen'].map(str),
        hoverinfo = 'location + text',
        locationmode = 'USA-states',
        colorscale = bpr.safe_tossup,
        showscale = False
    )
)

g.update_layout(
    title_text = 'U.S. Senate (R: ' + str(r) + ' D: ' + str(d) + ')',
    geo_scope = 'usa',
    dragmode = False,
)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#444CA0'], [0.5, '#F2DDA9'], [1.0, '#A04555']],
           …

In [443]:
widgets.VBox([widgets.VBox([vap, turnout, turnout_var, swing_vote_percentage, swing_impact])])

<hr />

In [444]:
pl.sign_in(username='jamesweichert', api_key='PUK0jPsKi3P0P9SVW2x1')

In [445]:
new_senate_elections = simulate_election(new_senate, rand=True)
nr, nd = check_composition(new_senate_elections)

In [446]:
h = go.FigureWidget(
    data = go.Choropleth(
        locations = new_senate_elections['State'],
        z = new_senate_elections['Senate_Scale'].astype(float),
        hovertext =  'R: ' + new_senate_elections['R_sen'].map(str) + ' D: ' + new_senate_elections['D_sen'].map(str),
        hoverinfo = 'location + text',
        locationmode = 'USA-states',
        colorscale = bpr.safe_likely_leans_tossup,
        showscale = False
    )
)

h.update_layout(
    title_text = 'U.S. Senate (R: ' + str(nr) + ' D: ' + str(nd) + ')',
    geo_scope = 'usa',
    dragmode = False
)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#444CA0'], [0.16666666666666666, '#487EBF'],
              …

In [447]:
pl.iplot(h, filename='NewSenatePartisan')